In [40]:
from fastlane_bot import Bot, Config, ConfigDB, ConfigNetwork, ConfigProvider
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC, CPCContainer, T, Pair
from fastlane_bot.tools.analyzer import CPCAnalyzer
from fastlane_bot.tools.optimizer import CPCArbOptimizer
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPCAnalyzer))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPCArbOptimizer))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(Bot))
from fastlane_bot.testing import *
import itertools as it
import collections as cl
plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("3.0", __VERSION__)

ConstantProductCurve v2.10.1 (07/May/2023)
CPCAnalyzer v0.1 (06/May/2023)
CPCArbOptimizer v3.6 (06/May/2023)
CarbonBot v3-b2.1 (03/May/2023)
Version = 3-b2.1 [requirements >= 3.0 is met]


/var/folders/rt/qnj8r6yd6131ccxkw_k9d9gc0000gn/T/ipykernel_66857/1971048585.py:12: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-dark')


# Mainnet Server [A011]

In [41]:
bot     = Bot()
CCm     = bot.get_curves()
CA      = CPCAnalyzer(CCm)

In [42]:
pairs0  = CA.CC.pairs(standardize=False)
pairs   = CA.pairs()
pairsc  = CA.pairsc()
tokens  = CA.tokens()

In [43]:
assert CA.pairs()  == CCm.pairs(standardize=True)
assert CA.pairsc() == {c.pairo.primary for c in CCm if c.P("exchange")=="carbon_v1"}
assert CA.tokens() == CCm.tokens()

## Overall market

In [44]:
print(f"Total pairs:    {len(pairs0):4}")
print(f"Primary pairs:  {len(pairs):4}")
print(f"...carbon:      {len(pairsc):4}")
print(f"Tokens:         {len(CCm.tokens()):4}")
print(f"Curves:         {len(CCm):4}")

Total pairs:    2864
Primary pairs:  2834
...carbon:        26
Tokens:         2233
Curves:         4156


## By pair

### All pairs

In [45]:
cbp0 = {pair: [c for c in CCm.bypairs(pair)] for pair in CCm.pairs()} # curves by (primary) pair
nbp0 = {pair: len(cc) for pair,cc in cbp0.items()}
assert len(cbp0) == len(CCm.pairs())
assert set(cbp0) == CCm.pairs()

### Only those with >1 curves

In [46]:
cbp = {pair: cc for pair, cc in cbp0.items() if len(cc)>1}
nbp = {pair: len(cc) for pair,cc in cbp.items()}
print(f"Pairs with >1 curves:  {len(cbp)}")
print(f"Curves in those:       {sum(nbp.values())}")
print(f"Average curves/pair:   {sum(nbp.values())/len(cbp):.1f}")

Pairs with >1 curves:  936
Curves in those:       2258
Average curves/pair:   2.4


### x=0 or y=0

In [47]:
xis0 = {c.cid: (c.x, c.y) for c in CCm if c.x==0}
yis0 = {c.cid: (c.x, c.y) for c in CCm if c.y==0}
assert len(xis0) == 0 # set loglevel debug to see removal of curves
assert len(yis0) == 0

### Prices

#### All

In [48]:
prices_da = {pair: 
             [(
                Pair.n(pair), pair, c.primaryp(), c.cid, c.cid[-8:], c.P("exchange"), c.tvl(tkn=pair.split("/")[0]),
                "x" if c.itm(cc) else "", c.buysell(verbose=False), c.buysell(verbose=True, withprice=True)
              ) for c in cc 
             ] 
             for pair, cc in cbp.items()
            }
#prices_da

#### Only for pairs that have at least on Carbon pair

In [49]:
prices_d = {pair: l for pair,l in prices_da.items() if pair in pairsc}
prices_l = list(it.chain(*prices_d.values()))

In [50]:
#prices_d

In [51]:
curves_by_pair = list(cl.Counter([r[1] for r in prices_l]).items())
curves_by_pair = sorted(curves_by_pair, key=lambda x: x[1], reverse=True)
curves_by_pair

[('WETH-6Cc2/USDC-eB48', 24),
 ('BNT-FF1C/WETH-6Cc2', 14),
 ('USDT-1ec7/USDC-eB48', 13),
 ('BNT-FF1C/vBNT-7f94', 12),
 ('WBTC-C599/WETH-6Cc2', 10),
 ('LINK-86CA/USDT-1ec7', 8),
 ('WETH-6Cc2/USDT-1ec7', 8),
 ('WETH-6Cc2/DAI-1d0F', 7),
 ('DAI-1d0F/USDC-eB48', 7),
 ('WBTC-C599/USDT-1ec7', 7),
 ('LINK-86CA/USDC-eB48', 6),
 ('DAI-1d0F/USDT-1ec7', 6),
 ('WBTC-C599/USDC-eB48', 6),
 ('BNT-FF1C/USDC-eB48', 5),
 ('rETH-6393/WETH-6Cc2', 5),
 ('CRV-cd52/USDC-eB48', 4),
 ('TSUKA-69eD/USDC-eB48', 4),
 ('stETH-fE84/WETH-6Cc2', 4),
 ('0x0-1AD5/WETH-6Cc2', 3),
 ('LYXe-be6D/USDC-eB48', 3),
 ('PEPE-1933/WETH-6Cc2', 2),
 ('ARB-4ad1/MATIC-eBB0', 2),
 ('SMT-7173/WETH-6Cc2', 2)]

In [52]:
CODE = """
# #### {pairn}

pair = "{pair}"
pricedf.loc[Pair.n(pair)]

pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)
"""

In [53]:
# for pair, _ in curves_by_pair:
#     print(CODE.format(pairn=Pair.n(pair), pair=pair))

#### Dataframe

In [54]:
pricedf0 = pd.DataFrame(prices_l, columns="pair,pairf,price,cid,cid0,exchange,vl,itm,bs,bsv".split(","))
pricedf = pricedf0.drop(['cid', 'pairf'], axis=1).sort_values(by=["pair", "exchange", "cid0"])
pricedf = pricedf.set_index(["pair", "exchange", "cid0"])
pricedf

price            vl itm  bs  \
pair       exchange   cid0                                       
0x0/WETH   carbon_v1  132277-0  0.000013  1.342084e+04       b   
                      132277-1  0.000015  3.597323e+02   x   s   
           uniswap_v2 551118da  0.000027  2.853407e+07   x  bs   
ARB/MATIC  carbon_v1  806240-0  1.507045  1.276054e+01       s   
                      806240-1  1.428571  1.418060e+02       b   
...                                  ...           ...  ..  ..   
rETH/WETH  uniswap_v3 ef318a34  1.067460  3.281094e-02   x  bs   
stETH/WETH carbon_v1  422914-0  1.010101  2.031521e-03       s   
                      422914-1  0.990099  8.011450e-02       b   
           uniswap_v2 ff7abe20  0.995957  2.457107e+03      bs   
           uniswap_v3 154a5023  0.997410  5.800729e+01      bs   

                                                                 bsv  
pair       exchange   cid0                                            
0x0/WETH   carbon_v1  132277-0           buy-0x0 @ 0.00 WETH per 0x0  
                      132277-1          sell-0x0 @ 0.00 WETH per 0x0  
           uniswap_v2 551118da      buy-sell-0x0 @ 0.00 WETH per 0x0  
ARB/MATIC  carbon_v1  806240-0         sell-ARB @ 1.51 MATIC per ARB  
                      806240-1          buy-ARB @ 1.43 MATIC per ARB  
...                                                              ...  
rETH/WETH  uniswap_v3 ef318a34    buy-sell-rETH @ 1.07 WETH per rETH  
stETH/WETH carbon_v1  422914-0      sell-stETH @ 1.01 WETH per stETH  
                      422914-1       buy-stETH @ 0.99 WETH per stETH  
           uniswap_v2 ff7abe20  buy-sell-stETH @ 1.00 WETH per stETH  
           uniswap_v3 154a5023  buy-sell-stETH @ 1.00 WETH per stETH  

[162 rows x 5 columns]

### Individual frames

#### WETH/USDC

In [55]:
pair = "WETH-6Cc2/USDC-eB48"
pricedf.loc[Pair.n(pair)]

price            vl itm  bs  \
exchange     cid0                                          
carbon_v1    057285-1  2099.999790      0.006040       s   
             057292-0  1853.408818      0.003314   x   b   
             057292-1  2000.000000      0.016387       s   
             057296-0  1929.999807      0.001033   x   b   
             057296-1  1949.999805     10.460391       s   
             057299-1  1940.000000      0.026117       s   
             057306-0  1405.000140      3.558719       b   
             057315-1  2300.000000      0.487950       s   
             057331-0  1800.000000      5.555556       b   
             057334-0  1700.000170      0.029412       b   
             057334-1  1999.999800      0.040000       s   
             057337-0  1850.000000      1.081081       b   
             057339-0  1800.000000      0.000556       b   
             057343-1  1989.999801      1.000000       s   
             057353-0  1854.000185      4.234699   x   b   
             057353-1  2047.999795      4.000000       s   
sushiswap_v2 803       1838.745520  17564.479610   x  bs   
             dd65c110  1841.913305  18392.882153   x  bs   
uniswap_v2   0c551073  1841.245125  32724.988098   x  bs   
             255       1840.773969  39241.200664   x  bs   
uniswap_v3   08cee9b5  1837.884991    210.709853   x  bs   
             346       1848.191535    233.930315   x  bs   
             593       1832.243200     58.054109   x  bs   
             76b13aa0  1843.328373    503.361718   x  bs   

                                                         bsv  
exchange     cid0                                             
carbon_v1    057285-1      sell-WETH @ 2100.00 USDC per WETH  
             057292-0       buy-WETH @ 1853.41 USDC per WETH  
             057292-1      sell-WETH @ 2000.00 USDC per WETH  
             057296-0       buy-WETH @ 1930.00 USDC per WETH  
             057296-1      sell-WETH @ 1950.00 USDC per WETH  
             057299-1      sell-WETH @ 1940.00 USDC per WETH  
             057306-0       buy-WETH @ 1405.00 USDC per WETH  
             057315-1      sell-WETH @ 2300.00 USDC per WETH  
             057331-0       buy-WETH @ 1800.00 USDC per WETH  
             057334-0       buy-WETH @ 1700.00 USDC per WETH  
             057334-1      sell-WETH @ 2000.00 USDC per WETH  
             057337-0       buy-WETH @ 1850.00 USDC per WETH  
             057339-0       buy-WETH @ 1800.00 USDC per WETH  
             057343-1      sell-WETH @ 1990.00 USDC per WETH  
             057353-0       buy-WETH @ 1854.00 USDC per WETH  
             057353-1      sell-WETH @ 2048.00 USDC per WETH  
sushiswap_v2 803       buy-sell-WETH @ 1838.75 USDC per WETH  
             dd65c110  buy-sell-WETH @ 1841.91 USDC per WETH  
uniswap_v2   0c551073  buy-sell-WETH @ 1841.25 USDC per WETH  
             255       buy-sell-WETH @ 1840.77 USDC per WETH  
uniswap_v3   08cee9b5  buy-sell-WETH @ 1837.88 USDC per WETH  
             346       buy-sell-WETH @ 1848.19 USDC per WETH  
             593       buy-sell-WETH @ 1832.24 USDC per WETH  
             76b13aa0  buy-sell-WETH @ 1843.33 USDC per WETH

In [56]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0], params=dict(verbose=False, debug=False))
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,USDC-eB48,WETH-6Cc2
593,41582.307951,-22.606771
255,56450.652563,-30.618946
6c988ffdc9e74acd97ccfb16dd65c110,21227.643365,-11.510354
803,34158.262754,-18.537726
00125d264f9d49369a467e7708cee9b5,91305.914678,-49.563418
1701411834604692317316873037158841057296-0,-1.994537,0.001033
346,-388246.353245,210.162583
1701411834604692317316873037158841057353-0,-7851.133636,4.234700
7ed16708962e459abe5431a176b13aa0,108176.797097,-58.665088
50ac5ace09c1483987af46c60c551073,43227.850692,-23.443872


In [57]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,USDC-eB48,WETH-6Cc2
593,41571.952013,-2.260116e+01
255,56415.726935,-3.060003e+01
6c988ffdc9e74acd97ccfb16dd65c110,21211.268185,-1.150149e+01
803,34142.638554,-1.852926e+01
00125d264f9d49369a467e7708cee9b5,91268.311093,-4.954305e+01
1701411834604692317316873037158841057296-0,-1.994537,1.033440e-03
346,-389081.069793,2.106146e+02
1701411834604692317316873037158841057353-0,-7851.133636,4.234700e+00
7ed16708962e459abe5431a176b13aa0,108176.797097,-5.866509e+01
50ac5ace09c1483987af46c60c551073,43198.720925,-2.342810e+01


#### BNT/WETH

In [58]:
pair = "BNT-FF1C/WETH-6Cc2"
pricedf.loc[Pair.n(pair)]

price            vl itm  bs  \
exchange   cid0                                       
bancor_v2  15838ce4  0.000237  1.219687e+07   x  bs   
           623       0.000238  1.218001e+07   x  bs   
bancor_v3  704       0.000237  3.182131e+07   x  bs   
carbon_v1  326030-0  0.000253  5.000000e+02       s   
           326030-1  0.000200  2.500000e+02       b   
           326031-0  0.000200  7.499999e+02       b   
           326031-1  0.002000  5.000000e+02       s   
           326034-0  0.000200  3.500000e+02       b   
           326034-1  0.002100  2.000000e+02       s   
           326076-0  0.000253  7.905138e+02   x   b   
           326077-0  0.000250  3.654723e-01   x   b   
           326077-1  0.000258  7.936392e+03       s   
uniswap_v2 290       0.000239  2.155118e+05   x  bs   
           38fd5e41  0.000235  2.172299e+05   x  bs   

                                                  bsv  
exchange   cid0                                        
bancor_v2  15838ce4  buy-sell-BNT @ 0.00 WETH per BNT  
           623       buy-sell-BNT @ 0.00 WETH per BNT  
bancor_v3  704       buy-sell-BNT @ 0.00 WETH per BNT  
carbon_v1  326030-0      sell-BNT @ 0.00 WETH per BNT  
           326030-1       buy-BNT @ 0.00 WETH per BNT  
           326031-0       buy-BNT @ 0.00 WETH per BNT  
           326031-1      sell-BNT @ 0.00 WETH per BNT  
           326034-0       buy-BNT @ 0.00 WETH per BNT  
           326034-1      sell-BNT @ 0.00 WETH per BNT  
           326076-0       buy-BNT @ 0.00 WETH per BNT  
           326077-0       buy-BNT @ 0.00 WETH per BNT  
           326077-1      sell-BNT @ 0.00 WETH per BNT  
uniswap_v2 290       buy-sell-BNT @ 0.00 WETH per BNT  
           38fd5e41  buy-sell-BNT @ 0.00 WETH per BNT

In [59]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,BNT-FF1C,WETH-6Cc2
290,374.609117,-8.914350e-02
623,5757.572044,-1.366639e+00
704,-3942.754081,9.347516e-01
30e4e04d729a43ccaf47743d38fd5e41,-483.143937,1.140629e-01
3743106036130323098097120681749450326076-0,811.641256,-2.000000e-01
9cd44a1425d34f07890193eb15838ce4,-2561.000640,6.070597e-01
3743106036130323098097120681749450326077-0,0.365472,-9.136815e-05
AMMIn,6944.187889,1.655874e+00
AMMOut,-6986.898658,-1.655874e+00
TOTAL NET,-42.710769,-1.532756e-10


In [60]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,BNT-FF1C,WETH-6Cc2
290,3.747722e+02,-0.089182
623,5.766767e+03,-1.368820
704,-3.918760e+03,0.929062
30e4e04d729a43ccaf47743d38fd5e41,-4.829808e+02,0.114024
3743106036130323098097120681749450326076-0,8.116413e+02,-0.200000
9cd44a1425d34f07890193eb15838ce4,-2.551805e+03,0.604879
3743106036130323098097120681749450326077-0,3.654725e-01,-0.000091
AMMIn,6.953546e+03,1.647965
AMMOut,-6.953546e+03,-1.658093
TOTAL NET,1.022236e-07,-0.010128


#### BNT/vBNT

In [61]:
pair = "BNT-FF1C/vBNT-7f94"
pricedf.loc[Pair.n(pair)]

price            vl itm  bs  \
exchange  cid0                                       
bancor_v2 653       1.333016  5.350297e+04   x  bs   
          e5f19b21  1.333016  5.350297e+04   x  bs   
bancor_v3 742       1.252518  2.466230e+06   x  bs   
carbon_v1 748950-0  0.940000  1.413879e+04       b   
          748965-1  0.909091  1.129751e+03       b   
          748966-1  1.000000  1.089256e+03       b   
          748976-0  1.331480  2.593859e+02       s   
          748976-1  1.111111  7.293739e+02   x   b   
          748977-0  1.428571  5.000000e+02       s   
          748977-1  1.250000  4.000000e+02   x   b   
          748990-0  1.052632  3.000000e+02   x   s   
          748990-1  0.952381  8.383963e+02       b   

                                                 bsv  
exchange  cid0                                        
bancor_v2 653       buy-sell-BNT @ 1.33 vBNT per BNT  
          e5f19b21  buy-sell-BNT @ 1.33 vBNT per BNT  
bancor_v3 742       buy-sell-BNT @ 1.25 vBNT per BNT  
carbon_v1 748950-0       buy-BNT @ 0.94 vBNT per BNT  
          748965-1       buy-BNT @ 0.91 vBNT per BNT  
          748966-1       buy-BNT @ 1.00 vBNT per BNT  
          748976-0      sell-BNT @ 1.33 vBNT per BNT  
          748976-1       buy-BNT @ 1.11 vBNT per BNT  
          748977-0      sell-BNT @ 1.43 vBNT per BNT  
          748977-1       buy-BNT @ 1.25 vBNT per BNT  
          748990-0      sell-BNT @ 1.05 vBNT per BNT  
          748990-1       buy-BNT @ 0.95 vBNT per BNT

In [62]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,BNT-FF1C,vBNT-7f94
653,814.443151,-1.053589e+03
cdae0b93cfa44a2195605c15e5f19b21,814.443151,-1.053589e+03
4423670769972200025023869896612986748990-0,-300.000000,3.244428e+02
742,-1421.680869,1.782736e+03
AMMIn,1628.886302,2.107179e+03
AMMOut,-1721.680869,-2.107179e+03
TOTAL NET,-92.794567,-8.982170e-09


In [63]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,BNT-FF1C,vBNT-7f94
653,8.164310e+02,-1056.084647
cdae0b93cfa44a2195605c15e5f19b21,8.164310e+02,-1056.084647
4423670769972200025023869896612986748990-0,-3.000000e+02,324.442842
742,-1.332862e+03,1671.239552
AMMIn,1.632862e+03,1995.682394
AMMOut,-1.632862e+03,-2112.169294
TOTAL NET,-3.034074e-08,-116.486899


#### USDT/USDC

In [64]:
pair = "USDT-1ec7/USDC-eB48"
pricedf.loc[Pair.n(pair)]

price            vl itm  bs  \
exchange     cid0                                       
carbon_v1    634371-0  1.005025  5.010000e+01       s   
             634371-1  0.995025  5.040025e+01   x   b   
             634391-0  1.001001  5.050000e+02       s   
             634391-1  1.000690  4.946550e+02   x   b   
sushiswap_v2 805       0.960965  1.885667e+03   x  bs   
             b22d879b  0.968776  1.878061e+03   x  bs   
uniswap_v2   246       0.998090  2.373632e+07   x  bs   
             9b269168  1.001828  2.366179e+07   x  bs   
uniswap_v3   2616d525  1.002654  3.514249e+03   x  bs   
             357       1.002654  3.514249e+03   x  bs   
             486       1.000655  2.366083e+06   x  bs   
             b7be04bd  1.000783  6.273568e+06   x  bs   
             d42fa3d0  1.000675  6.048697e+06   x  bs   

                                                      bsv  
exchange     cid0                                          
carbon_v1    634371-0      sell-USDT @ 1.01 USDC per USDT  
             634371-1       buy-USDT @ 1.00 USDC per USDT  
             634391-0      sell-USDT @ 1.00 USDC per USDT  
             634391-1       buy-USDT @ 1.00 USDC per USDT  
sushiswap_v2 805       buy-sell-USDT @ 0.96 USDC per USDT  
             b22d879b  buy-sell-USDT @ 0.97 USDC per USDT  
uniswap_v2   246       buy-sell-USDT @ 1.00 USDC per USDT  
             9b269168  buy-sell-USDT @ 1.00 USDC per USDT  
uniswap_v3   2616d525  buy-sell-USDT @ 1.00 USDC per USDT  
             357       buy-sell-USDT @ 1.00 USDC per USDT  
             486       buy-sell-USDT @ 1.00 USDC per USDT  
             b7be04bd  buy-sell-USDT @ 1.00 USDC per USDT  
             d42fa3d0  buy-sell-USDT @ 1.00 USDC per USDT

In [65]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,USDT-1ec7,USDC-eB48
357,1157.643881,-1159.571540
246,-15349.026041,15339.553785
486,-532123.253410,532477.537949
805,-18.897843,18.531599
a3c4f16e3fd042ab93650511b22d879b,-15.089304,14.856886
c9a1ba7537f242ecacf31755b7be04bd,664337.575800,-664822.553961
1cc430ac74f7494cb011948f9b269168,6799.232088,-6807.745655
15c7994e2d2c468fbad5fb782616d525,1157.643881,-1159.571540
4c50c9e4fdde4aefbf495b30d42fa3d0,-126020.483414,126105.673706
1020847100762815390390123822295304634391-1,6.706633,-6.711219


In [66]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,USDT-1ec7,USDC-eB48
357,1157.644322,-1159.571981
246,-15349.021589,15339.549330
486,-532105.476410,532459.748909
805,-18.897842,18.531599
a3c4f16e3fd042ab93650511b22d879b,-15.089304,14.856886
c9a1ba7537f242ecacf31755b7be04bd,664342.290316,-664827.271669
1cc430ac74f7494cb011948f9b269168,6799.236535,-6807.750104
15c7994e2d2c468fbad5fb782616d525,1157.644322,-1159.571981
4c50c9e4fdde4aefbf495b30d42fa3d0,-125975.037369,126060.196884
1020847100762815390390123822295304634391-1,6.707008,-6.711594


#### WBTC/WETH

In [67]:
pair = "WBTC-C599/WETH-6Cc2"
pricedf.loc[Pair.n(pair)]

price          vl itm  bs  \
exchange     cid0                                      
carbon_v1    709362-0  15.399750    0.133758       s   
             709362-1  14.285714    0.417087       b   
sushiswap_v2 804       14.999746  779.023840      bs   
             eee68882  14.980919  779.513941      bs   
uniswap_v3   23c8e88d  15.097158    0.127246      bs   
             3d1aaf34  14.992129  209.185313      bs   
             466       15.097158    0.127246      bs   
             478       15.105116    0.000017      bs   
             4e3d56b9  15.023596    0.000017      bs   
             edc87668  14.974717   17.775987      bs   

                                                       bsv  
exchange     cid0                                           
carbon_v1    709362-0      sell-WBTC @ 15.40 WETH per WBTC  
             709362-1       buy-WBTC @ 14.29 WETH per WBTC  
sushiswap_v2 804       buy-sell-WBTC @ 15.00 WETH per WBTC  
             eee68882  buy-sell-WBTC @ 14.98 WETH per WBTC  
uniswap_v3   23c8e88d  buy-sell-WBTC @ 15.10 WETH per WBTC  
             3d1aaf34  buy-sell-WBTC @ 14.99 WETH per WBTC  
             466       buy-sell-WBTC @ 15.10 WETH per WBTC  
             478       buy-sell-WBTC @ 15.11 WETH per WBTC  
             4e3d56b9  buy-sell-WBTC @ 15.02 WETH per WBTC  
             edc87668  buy-sell-WBTC @ 14.97 WETH per WBTC

In [68]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,WBTC-C599,WETH-6Cc2
466,0.010511,-1.585580e-01
804,0.148015,-2.219342e+00
478,0.000012,-1.831884e-04
b1715f4832ae4a1dabde9cbe4e3d56b9,0.000010,-1.527214e-04
0a1e0af654974dd6848b52233d1aaf34,8.794744,-1.318353e+02
dfefc9ca163747858dc6b79eeee68882,-0.096668,1.448531e+00
edb7550782154a5b8eb1e4feedc87668,-8.874319,1.329236e+02
652229ad70bd4d19b9d1420123c8e88d,0.010511,-1.585580e-01
AMMIn,8.963804,1.343721e+02
AMMOut,-8.970987,-1.343721e+02


In [69]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,WBTC-C599,WETH-6Cc2
466,1.051115e-02,-0.158558
804,1.480545e-01,-2.219936
478,1.212800e-05,-0.000183
b1715f4832ae4a1dabde9cbe4e3d56b9,1.016574e-05,-0.000153
0a1e0af654974dd6848b52233d1aaf34,8.801849e+00,-131.941815
dfefc9ca163747858dc6b79eeee68882,-9.662812e-02,1.447937
edb7550782154a5b8eb1e4feedc87668,-8.874319e+00,132.923598
652229ad70bd4d19b9d1420123c8e88d,1.051115e-02,-0.158558
AMMIn,8.970948e+00,134.371534
AMMOut,-8.970948e+00,-134.479203


#### LINK/USDT

In [70]:
pair = "LINK-86CA/USDT-1ec7"
pricedf.loc[Pair.n(pair)]

price         vl itm  bs  \
exchange     cid0                                    
carbon_v1    960408-0  6.900402   0.055841   x   b   
             960408-1  7.700000  37.987504       s   
sushiswap_v2 2dc48f37  7.123545  75.100134   x  bs   
             791       7.123545  75.100134   x  bs   
uniswap_v2   171       7.328775  65.660409   x  bs   
             8b9e0410  6.215835  71.305185   x  bs   
uniswap_v3   549       6.277183   3.483579   x  bs   
             984a84b8  6.828776   3.339705   x  bs   

                                                      bsv  
exchange     cid0                                          
carbon_v1    960408-0       buy-LINK @ 6.90 USDT per LINK  
             960408-1      sell-LINK @ 7.70 USDT per LINK  
sushiswap_v2 2dc48f37  buy-sell-LINK @ 7.12 USDT per LINK  
             791       buy-sell-LINK @ 7.12 USDT per LINK  
uniswap_v2   171       buy-sell-LINK @ 7.33 USDT per LINK  
             8b9e0410  buy-sell-LINK @ 6.22 USDT per LINK  
uniswap_v3   549       buy-sell-LINK @ 6.28 USDT per LINK  
             984a84b8  buy-sell-LINK @ 6.83 USDT per LINK

In [71]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,LINK-86CA,USDT-1ec7
791,0.783228,-5.465359e+00
549,-0.807262,5.070853e+00
171,1.164136,-8.239524e+00
661854190b4b461b840ac93c8b9e0410,-1.654214,1.078262e+01
4763953136893138488487244504044754960408-0,0.055843,-3.853270e-01
d6804f9f95a64ea586c0aab32dc48f37,0.783228,-5.465359e+00
539a301956344755aef66083984a84b8,-0.541869,3.702100e+00
AMMIn,2.786434,1.955557e+01
AMMOut,-3.003345,-1.955557e+01
TOTAL NET,-0.216911,-1.379777e-08


In [72]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,LINK-86CA,USDT-1ec7
791,7.898342e-01,-5.510510
549,-8.072620e-01,5.070853
171,1.169995e+00,-8.279565
661854190b4b461b840ac93c8b9e0410,-1.648355e+00,10.742570
4763953136893138488487244504044754960408-0,5.584286e-02,-0.385327
d6804f9f95a64ea586c0aab32dc48f37,7.898342e-01,-5.510510
539a301956344755aef66083984a84b8,-3.498898e-01,2.390069
AMMIn,2.805506e+00,18.203492
AMMOut,-2.805506e+00,-19.685913
TOTAL NET,-2.131628e-13,-1.482421


#### WBTC/USDT

In [73]:
pair = "WBTC-C599/USDT-1ec7"
pricedf.loc[Pair.n(pair)]

price        vl itm  bs  \
exchange     cid0                                       
carbon_v1    920820-1  29500.000000  0.000065       s   
sushiswap_v2 814       29043.429150  0.031191   x  bs   
             b7ad4378  29043.429150  0.031191   x  bs   
uniswap_v2   183       28257.884111  0.290096   x  bs   
             44235983  27609.205207  0.460800   x  bs   
uniswap_v3   472c5f6c  27602.349628  4.963284   x  bs   
             da663695  27654.095998  0.015273   x  bs   

                                                          bsv  
exchange     cid0                                              
carbon_v1    920820-1      sell-WBTC @ 29500.00 USDT per WBTC  
sushiswap_v2 814       buy-sell-WBTC @ 29043.43 USDT per WBTC  
             b7ad4378  buy-sell-WBTC @ 29043.43 USDT per WBTC  
uniswap_v2   183       buy-sell-WBTC @ 28257.88 USDT per WBTC  
             44235983  buy-sell-WBTC @ 27609.21 USDT per WBTC  
uniswap_v3   472c5f6c  buy-sell-WBTC @ 27602.35 USDT per WBTC  
             da663695  buy-sell-WBTC @ 27654.10 USDT per WBTC

In [74]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,WBTC-C599,USDT-1ec7
183,0.001711,-4.779064e+01
814,0.000402,-1.137699e+01
8b84d53e2a1f47d7aa2e021b472c5f6c,-0.012376,3.416171e+02
9dab98b0ea8c47a5bb17728844235983,0.000027,-7.422818e-01
1a63b94c4aab468aadf132f6b7ad4378,0.000402,-1.137699e+01
6133b4b856974dbcbbaeafd0da663695,0.009779,-2.703302e+02
AMMIn,0.012320,3.416171e+02
AMMOut,-0.012376,-3.416171e+02
TOTAL NET,-0.000056,2.672739e-07


In [75]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,WBTC-C599,USDT-1ec7
183,1.711189e-03,-47.790774
814,4.018163e-04,-11.377000
8b84d53e2a1f47d7aa2e021b472c5f6c,-1.232026e-02,340.070657
9dab98b0ea8c47a5bb17728844235983,2.689623e-05,-0.742497
1a63b94c4aab468aadf132f6b7ad4378,4.018163e-04,-11.377000
6133b4b856974dbcbbaeafd0da663695,9.778542e-03,-270.330222
AMMIn,1.232026e-02,340.070657
AMMOut,-1.232026e-02,-341.617493
TOTAL NET,-1.367952e-11,-1.546836


#### BNT/USDC

In [76]:
pair = "BNT-FF1C/USDC-eB48"
pricedf.loc[Pair.n(pair)]

price            vl itm  bs  \
exchange  cid0                                       
bancor_v2 2820a318  0.430096  1.560855e+06   x  bs   
          652       0.439371  1.544040e+06   x  bs   
bancor_v3 720       0.440302  5.223673e+06   x  bs   
carbon_v1 480199-0  2.000000  2.910000e+01       s   
          480202-1  1.480041  4.247463e+04       s   

                                                 bsv  
exchange  cid0                                        
bancor_v2 2820a318  buy-sell-BNT @ 0.43 USDC per BNT  
          652       buy-sell-BNT @ 0.44 USDC per BNT  
bancor_v3 720       buy-sell-BNT @ 0.44 USDC per BNT  
carbon_v1 480199-0      sell-BNT @ 2.00 USDC per BNT  
          480202-1      sell-BNT @ 1.48 USDC per BNT

In [77]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,BNT-FF1C,USDC-eB48
652,1008.424079,-4.424948e+02
720,6180.961776,-2.715067e+03
97327fc2000549f98a6e6c812820a318,-7273.111432,3.157562e+03
AMMIn,7189.385855,3.157562e+03
AMMOut,-7273.111432,-3.157562e+03
TOTAL NET,-83.725577,-1.979060e-09


In [78]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,BNT-FF1C,USDC-eB48
652,1.023967e+03,-449.305769
720,6.233600e+03,-2738.134315
97327fc2000549f98a6e6c812820a318,-7.257566e+03,3150.750108
AMMIn,7.257566e+03,3150.750108
AMMOut,-7.257566e+03,-3187.440084
TOTAL NET,-4.190952e-09,-36.689976


#### WETH/DAI

In [79]:
pair = "WETH-6Cc2/DAI-1d0F"
pricedf.loc[Pair.n(pair)]

price           vl itm  bs  \
exchange     cid0                                         
carbon_v1    211457-1  1944.999806     0.001000       s   
sushiswap_v2 817       1838.100731  4281.600643      bs   
             f48bfe9f  1844.936214  4271.976508      bs   
uniswap_v3   01933a67  1840.121649     8.466599      bs   
             28a6fe54  1849.440070    10.797647      bs   
             594       1840.121649     8.466599      bs   
             7b8cd45b  1844.933284   117.246807      bs   

                                                        bsv  
exchange     cid0                                            
carbon_v1    211457-1      sell-WETH @ 1945.00 DAI per WETH  
sushiswap_v2 817       buy-sell-WETH @ 1838.10 DAI per WETH  
             f48bfe9f  buy-sell-WETH @ 1844.94 DAI per WETH  
uniswap_v3   01933a67  buy-sell-WETH @ 1840.12 DAI per WETH  
             28a6fe54  buy-sell-WETH @ 1849.44 DAI per WETH  
             594       buy-sell-WETH @ 1840.12 DAI per WETH  
             7b8cd45b  buy-sell-WETH @ 1844.93 DAI per WETH

In [80]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,DAI-1d0F,WETH-6Cc2
594,1594.209278,-0.865479
817,6804.141181,-3.695334
2b3bf6093e55415785a319a6f48bfe9f,-505.571508,0.274067
a6595d66f70c432a9b68557428a6fe54,-286.288242,0.154798
644ebe18973a4af2aea93d9401933a67,1594.209278,-0.865479
074a75bbb19a4ef9a0b583317b8cd45b,-9200.700113,4.987646
AMMIn,9992.559736,5.416511
AMMOut,-9992.559863,-5.426292
TOTAL NET,-0.000127,-0.009781


In [81]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,DAI-1d0F,WETH-6Cc2
594,1594.209278,-8.654788e-01
817,6803.252704,-3.694852e+00
2b3bf6093e55415785a319a6f48bfe9f,-506.459634,2.745486e-01
a6595d66f70c432a9b68557428a6fe54,-286.288242,1.547984e-01
644ebe18973a4af2aea93d9401933a67,1594.209278,-8.654788e-01
074a75bbb19a4ef9a0b583317b8cd45b,-9216.963432,4.996463e+00
AMMIn,9991.671260,5.425810e+00
AMMOut,-10009.711308,-5.425810e+00
TOTAL NET,-18.040048,-8.219217e-08


#### DAI/USDT

In [82]:
pair = "DAI-1d0F/USDT-1ec7"
pricedf.loc[Pair.n(pair)]

price            vl itm  bs  \
exchange     cid0                                       
carbon_v1    268742-0  0.995000     20.100501       b   
             268742-1  1.005000     20.000000       s   
sushiswap_v2 795       0.994182    380.253948      bs   
             eb9230ec  0.994182    380.253948      bs   
uniswap_v3   7b12cc08  0.999172  15514.318187      bs   
             a21da88e  0.999426  50235.221857      bs   

                                                    bsv  
exchange     cid0                                        
carbon_v1    268742-0       buy-DAI @ 1.00 USDT per DAI  
             268742-1      sell-DAI @ 1.00 USDT per DAI  
sushiswap_v2 795       buy-sell-DAI @ 0.99 USDT per DAI  
             eb9230ec  buy-sell-DAI @ 0.99 USDT per DAI  
uniswap_v3   7b12cc08  buy-sell-DAI @ 1.00 USDT per DAI  
             a21da88e  buy-sell-DAI @ 1.00 USDT per DAI

In [83]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,USDT-1ec7,DAI-1d0F
795,4.972032e-01,-0.498801
de1e0609fbe84008b5bed757a21da88e,-3.577997e+03,3580.063945
c58b09b0d6bc4534baf324267b12cc08,3.577003e+03,-3579.552900
4c85f66495b144368500942aeb9230ec,4.972032e-01,-0.498801
AMMIn,3.577997e+03,3580.063945
AMMOut,-3.577997e+03,-3580.550502
TOTAL NET,1.869082e-07,-0.486557


In [84]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,USDT-1ec7,DAI-1d0F
795,0.497203,-4.988007e-01
de1e0609fbe84008b5bed757a21da88e,-3578.483445,3.580551e+03
c58b09b0d6bc4534baf324267b12cc08,3577.002764,-3.579553e+03
4c85f66495b144368500942aeb9230ec,0.497203,-4.988007e-01
AMMIn,3577.997171,3.580551e+03
AMMOut,-3578.483445,-3.580551e+03
TOTAL NET,-0.486274,2.228529e-08


#### DAI/USDC

In [85]:
pair = "DAI-1d0F/USDC-eB48"
pricedf.loc[Pair.n(pair)]

price            vl itm  bs  \
exchange     cid0                                       
carbon_v1    845828-0  1.001001  3.000000e+01       s   
             845828-1  0.999001  2.002000e+01       b   
sushiswap_v2 839       0.995940  2.573876e+04      bs   
             de6b2675  0.999022  2.569909e+04      bs   
uniswap_v3   35fe6796  1.002196  1.291845e+02      bs   
             3880de01  1.000395  3.107288e+07      bs   
             8a01aa74  1.000078  2.687145e+07      bs   

                                                    bsv  
exchange     cid0                                        
carbon_v1    845828-0      sell-DAI @ 1.00 USDC per DAI  
             845828-1       buy-DAI @ 1.00 USDC per DAI  
sushiswap_v2 839       buy-sell-DAI @ 1.00 USDC per DAI  
             de6b2675  buy-sell-DAI @ 1.00 USDC per DAI  
uniswap_v3   35fe6796  buy-sell-DAI @ 1.00 USDC per DAI  
             3880de01  buy-sell-DAI @ 1.00 USDC per DAI  
             8a01aa74  buy-sell-DAI @ 1.00 USDC per DAI

In [86]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,USDC-eB48,DAI-1d0F
839,2.727103e+01,-2.732406e+01
873a7e371be64eb485a3cfee35fe6796,-4.342041e+01,4.336886e+01
7af1ca9ab5eb4b5f98105df03880de01,-6.589934e+06,6.588031e+06
56ae8dd6f6e44916af9b037ade6b2675,7.456878e+00,-7.459846e+00
21f3ea686abd44c6b7829e488a01aa74,6.589943e+06,-6.589347e+06
AMMIn,6.589978e+06,6.588075e+06
AMMOut,-6.589978e+06,-6.589382e+06
TOTAL NET,4.082918e-06,-1.307515e+03


In [87]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,USDC-eB48,DAI-1d0F
839,2.727076e+01,-2.732379e+01
873a7e371be64eb485a3cfee35fe6796,-4.342132e+01,4.336977e+01
7af1ca9ab5eb4b5f98105df03880de01,-6.591242e+06,6.589339e+06
56ae8dd6f6e44916af9b037ade6b2675,7.456608e+00,-7.459575e+00
21f3ea686abd44c6b7829e488a01aa74,6.589943e+06,-6.589347e+06
AMMIn,6.589978e+06,6.589382e+06
AMMOut,-6.591285e+06,-6.589382e+06
TOTAL NET,-1.307754e+03,-3.297813e-06


#### WETH/USDT

In [88]:
pair = "WETH-6Cc2/USDT-1ec7"
pricedf.loc[Pair.n(pair)]

price            vl itm  bs  \
exchange     cid0                                          
carbon_v1    691656-0  1891.000189      0.002644   x   b   
sushiswap_v2 840       1837.198565  10784.561352   x  bs   
             8f44c4dd  1841.824009  10761.574447   x  bs   
uniswap_v2   256       1841.296145  32753.117769   x  bs   
             f9829ce2  1841.209489  33265.845470   x  bs   
uniswap_v3   406a386b  1838.536133    199.303088   x  bs   
             7d051d19  1838.766710     48.706142   x  bs   
             b23b929d  1841.759896    292.510987   x  bs   

                                                         bsv  
exchange     cid0                                             
carbon_v1    691656-0       buy-WETH @ 1891.00 USDT per WETH  
sushiswap_v2 840       buy-sell-WETH @ 1837.20 USDT per WETH  
             8f44c4dd  buy-sell-WETH @ 1841.82 USDT per WETH  
uniswap_v2   256       buy-sell-WETH @ 1841.30 USDT per WETH  
             f9829ce2  buy-sell-WETH @ 1841.21 USDT per WETH  
uniswap_v3   406a386b  buy-sell-WETH @ 1838.54 USDT per WETH  
             7d051d19  buy-sell-WETH @ 1838.77 USDT per WETH  
             b23b929d  buy-sell-WETH @ 1841.76 USDT per WETH

In [89]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,USDT-1ec7,WETH-6Cc2
840,10003.618548,-5.439547
256,-3155.585199,1.713964
f8c64dd2abd24ca9941c4407f9829ce2,-2484.245099,1.349356
a838c700fdba49e59bc0376b406a386b,23733.164297,-12.900400
1f595b1af09543b6959e9f1cb23b929d,-41440.790472,22.505836
2722258935367507707706996859454145691656-0,-5.000000,0.002644
ac864c78bea34cb7b5e51b038f44c4dd,-2457.231766,1.334460
67f9d1e2b3fc407eb44dcb637d051d19,15806.069575,-8.595259
AMMIn,49542.852420,26.906261
AMMOut,-49542.852537,-26.935206


In [90]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,USDT-1ec7,WETH-6Cc2
840,10001.839956,-5.438580e+00
256,-3160.992871,1.716902e+00
f8c64dd2abd24ca9941c4407f9829ce2,-2489.737295,1.352339e+00
a838c700fdba49e59bc0376b406a386b,23726.566675,-1.289682e+01
1f595b1af09543b6959e9f1cb23b929d,-41473.022431,2.252334e+01
2722258935367507707706996859454145691656-0,-5.000000,2.644104e-03
ac864c78bea34cb7b5e51b038f44c4dd,-2459.008800,1.335426e+00
67f9d1e2b3fc407eb44dcb637d051d19,15806.069575,-8.595259e+00
AMMIn,49534.476206,2.693066e+01
AMMOut,-49587.761398,-2.693066e+01


#### LINK/USDC

In [91]:
pair = "LINK-86CA/USDC-eB48"
pricedf.loc[Pair.n(pair)]

price          vl itm  bs  \
exchange     cid0                                     
carbon_v1    497903-1  7.750000  342.883761       s   
sushiswap_v2 792       6.935941    3.543424   x  bs   
             e6e5be6d  6.935941    3.543424   x  bs   
uniswap_v2   176       6.794102  183.739077   x  bs   
             ae494094  6.794102  183.739077   x  bs   
uniswap_v3   51505935  6.546066  295.418600   x  bs   

                                                      bsv  
exchange     cid0                                          
carbon_v1    497903-1      sell-LINK @ 7.75 USDC per LINK  
sushiswap_v2 792       buy-sell-LINK @ 6.94 USDC per LINK  
             e6e5be6d  buy-sell-LINK @ 6.94 USDC per LINK  
uniswap_v2   176       buy-sell-LINK @ 6.79 USDC per LINK  
             ae494094  buy-sell-LINK @ 6.79 USDC per LINK  
uniswap_v3   51505935  buy-sell-LINK @ 6.55 USDC per LINK

In [92]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,LINK-86CA,USDC-eB48
176,1.720896,-1.147696e+01
792,0.051931,-3.499304e-01
368ffa4c667d4eb48fe33873e6e5be6d,0.051931,-3.499304e-01
83ed42e38ac3464482db3af8ae494094,1.720896,-1.147696e+01
f0e05bc5af5546cda90e559151505935,-3.613302,2.365378e+01
AMMIn,3.545654,2.365378e+01
AMMOut,-3.613302,-2.365378e+01
TOTAL NET,-0.067648,2.557954e-13


In [93]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,LINK-86CA,USDC-eB48
176,1.720960e+00,-11.477379
792,5.193180e-02,-0.349939
368ffa4c667d4eb48fe33873e6e5be6d,5.193180e-02,-0.349939
83ed42e38ac3464482db3af8ae494094,1.720960e+00,-11.477379
f0e05bc5af5546cda90e559151505935,-3.545784e+00,23.211772
AMMIn,3.545784e+00,23.211772
AMMOut,-3.545784e+00,-23.654634
TOTAL NET,2.397726e-11,-0.442862


#### PEPE/WETH

In [94]:
pair = "PEPE-1933/WETH-6Cc2"
pricedf.loc[Pair.n(pair)]

price            vl itm bs  \
exchange  cid0                                          
carbon_v1 440620-1  4.000000e-07  7.144675e+06      s   
          440621-1  4.500000e-07  1.315789e+06      s   

                                               bsv  
exchange  cid0                                      
carbon_v1 440620-1  sell-PEPE @ 0.00 WETH per PEPE  
          440621-1  sell-PEPE @ 0.00 WETH per PEPE

In [95]:
# pi = CA.pair_data(pair)
# O = CPCArbOptimizer(pi.CC)
# r = O.margp_optimizer(pair.split("/")[0])
# r.trade_instructions(ti_format=O.TIF_DFAGGR)

In [96]:
# r = O.margp_optimizer(pair.split("/")[1])
# r.trade_instructions(ti_format=O.TIF_DFAGGR)

#### stETH/WETH

In [97]:
pair = "stETH-fE84/WETH-6Cc2"
pricedf.loc[Pair.n(pair)]

price           vl itm  bs  \
exchange   cid0                                      
carbon_v1  422914-0  1.010101     0.002032       s   
           422914-1  0.990099     0.080114       b   
uniswap_v2 ff7abe20  0.995957  2457.106537      bs   
uniswap_v3 154a5023  0.997410    58.007285      bs   

                                                      bsv  
exchange   cid0                                            
carbon_v1  422914-0      sell-stETH @ 1.01 WETH per stETH  
           422914-1       buy-stETH @ 0.99 WETH per stETH  
uniswap_v2 ff7abe20  buy-sell-stETH @ 1.00 WETH per stETH  
uniswap_v3 154a5023  buy-sell-stETH @ 1.00 WETH per stETH

In [98]:
# pi = CA.pair_data(pair)
# O = CPCArbOptimizer(pi.CC)
# r = O.margp_optimizer(pair.split("/")[0])
# r.trade_instructions(ti_format=O.TIF_DFAGGR)

In [99]:
# r = O.margp_optimizer(pair.split("/")[1])
# r.trade_instructions(ti_format=O.TIF_DFAGGR)

#### rETH/WETH

In [100]:
pair = "rETH-6393/WETH-6Cc2"
pricedf.loc[Pair.n(pair)]

price          vl itm  bs  \
exchange     cid0                                     
carbon_v1    903115-0  1.069000    1.870907       b   
sushiswap_v2 833       1.237861    0.000312   x  bs   
             b845b79c  1.237861    0.000312   x  bs   
uniswap_v3   82c4849c  1.070096  301.692416   x  bs   
             ef318a34  1.067460    0.032811   x  bs   

                                                      bsv  
exchange     cid0                                          
carbon_v1    903115-0       buy-rETH @ 1.07 WETH per rETH  
sushiswap_v2 833       buy-sell-rETH @ 1.24 WETH per rETH  
             b845b79c  buy-sell-rETH @ 1.24 WETH per rETH  
uniswap_v3   82c4849c  buy-sell-rETH @ 1.07 WETH per rETH  
             ef318a34  buy-sell-rETH @ 1.07 WETH per rETH

In [101]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,rETH-6393,WETH-6Cc2
833,0.000012,-1.354606e-05
165d9de4a5b74ea1aa09e2c3b845b79c,0.000012,-1.354606e-05
287994fd2a0e4bf4887ed0f282c4849c,0.003877,-4.148734e-03
9c5bf9045ed04c80897ae947ef318a34,-0.003911,4.175826e-03
AMMIn,0.003901,4.175826e-03
AMMOut,-0.003911,-4.175826e-03
TOTAL NET,-0.000010,-5.605009e-11


In [102]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,rETH-6393,WETH-6Cc2
833,1.176971e-05,-0.000014
165d9de4a5b74ea1aa09e2c3b845b79c,1.176971e-05,-0.000014
287994fd2a0e4bf4887ed0f282c4849c,3.886990e-03,-0.004159
9c5bf9045ed04c80897ae947ef318a34,-3.910529e-03,0.004176
AMMIn,3.910529e-03,0.004176
AMMOut,-3.910529e-03,-0.004187
TOTAL NET,-2.411486e-12,-0.000011


#### ARB/MATIC

In [103]:
pair = "ARB-4ad1/MATIC-eBB0"
pricedf.loc[Pair.n(pair)]

price          vl itm bs                            bsv
exchange  cid0                                                                
carbon_v1 806240-0  1.507045   12.760538      s  sell-ARB @ 1.51 MATIC per ARB
          806240-1  1.428571  141.806023      b   buy-ARB @ 1.43 MATIC per ARB

In [104]:
# pi = CA.pair_data(pair)
# O = CPCArbOptimizer(pi.CC)
# r = O.margp_optimizer(pair.split("/")[0])
# r.trade_instructions(ti_format=O.TIF_DFAGGR)

In [105]:
# r = O.margp_optimizer(pair.split("/")[1])
# r.trade_instructions(ti_format=O.TIF_DFAGGR)

#### 0x0/WETH

In [106]:
pair = "0x0-1AD5/WETH-6Cc2"
pricedf.loc[Pair.n(pair)]

price            vl itm  bs  \
exchange   cid0                                       
carbon_v1  132277-0  0.000013  1.342084e+04       b   
           132277-1  0.000015  3.597323e+02   x   s   
uniswap_v2 551118da  0.000027  2.853407e+07   x  bs   

                                                  bsv  
exchange   cid0                                        
carbon_v1  132277-0       buy-0x0 @ 0.00 WETH per 0x0  
           132277-1      sell-0x0 @ 0.00 WETH per 0x0  
uniswap_v2 551118da  buy-sell-0x0 @ 0.00 WETH per 0x0

In [107]:
# pi = CA.pair_data(pair)
# O = CPCArbOptimizer(pi.CC)
# r = O.margp_optimizer(pair.split("/")[0])
# r.trade_instructions(ti_format=O.TIF_DFAGGR)

In [108]:
# r = O.margp_optimizer(pair.split("/")[1])
# r.trade_instructions(ti_format=O.TIF_DFAGGR)

#### TSUKA/USDC

In [109]:
pair = "TSUKA-69eD/USDC-eB48"
pricedf.loc[Pair.n(pair)]

price            vl itm  bs  \
exchange   cid0                                       
carbon_v1  017697-1  0.120000  4.656734e+04       s   
uniswap_v2 254       0.044583  3.690614e+07   x  bs   
           feeb42d8  0.045679  3.647798e+07   x  bs   
uniswap_v3 f0641f2c  0.049762  2.150701e+04   x  bs   

                                                      bsv  
exchange   cid0                                            
carbon_v1  017697-1      sell-TSUKA @ 0.12 USDC per TSUKA  
uniswap_v2 254       buy-sell-TSUKA @ 0.04 USDC per TSUKA  
           feeb42d8  buy-sell-TSUKA @ 0.05 USDC per TSUKA  
uniswap_v3 f0641f2c  buy-sell-TSUKA @ 0.05 USDC per TSUKA

In [110]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,USDC-eB48,TSUKA-69eD
254,5.327878e+03,-118735.423197
dab521a0a0134603b0451582feeb42d8,-4.725996e+03,104050.794662
3aa86ba4746c48f6bea2b713f0641f2c,-6.018815e+02,12163.370342
AMMIn,5.327878e+03,116214.165004
AMMOut,-5.327878e+03,-118735.423197
TOTAL NET,3.637979e-10,-2521.258193


In [111]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,USDC-eB48,TSUKA-69eD
254,5270.962094,-1.174751e+05
dab521a0a0134603b0451582feeb42d8,-4782.938935,1.053117e+05
3aa86ba4746c48f6bea2b713f0641f2c,-601.881511,1.216337e+04
AMMIn,5270.962094,1.174751e+05
AMMOut,-5384.820446,-1.174751e+05
TOTAL NET,-113.858352,4.656613e-09


#### WBTC/USDC

In [112]:
pair = "WBTC-C599/USDC-eB48"
pricedf.loc[Pair.n(pair)]

price         vl itm  bs  \
exchange   cid0                                        
carbon_v1  537493-0  28000.000000   0.045365   x   b   
uniswap_v2 02248661  27658.003470   6.152326   x  bs   
           332       27658.003470   6.152326   x  bs   
uniswap_v3 1aaa6fb7  27543.399583   0.104690   x  bs   
           56caf843  27776.038525   0.093606      bs   
           cf72417e  27549.422647  22.041670   x  bs   

                                                        bsv  
exchange   cid0                                              
carbon_v1  537493-0       buy-WBTC @ 28000.00 USDC per WBTC  
uniswap_v2 02248661  buy-sell-WBTC @ 27658.00 USDC per WBTC  
           332       buy-sell-WBTC @ 27658.00 USDC per WBTC  
uniswap_v3 1aaa6fb7  buy-sell-WBTC @ 27543.40 USDC per WBTC  
           56caf843  buy-sell-WBTC @ 27776.04 USDC per WBTC  
           cf72417e  buy-sell-WBTC @ 27549.42 USDC per WBTC

In [113]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,WBTC-C599,USDC-eB48
332,0.006041,-1.667642e+02
4b7ef4b0433441ad854c728302248661,0.006041,-1.667642e+02
8ef933e3eef94f8ea2a73cc11aaa6fb7,-0.023895,6.582328e+02
d7bd6d60f8f14492a9c5587a56caf843,0.033098,-9.161023e+02
8748e770adfe49f78064fd9bcf72417e,-0.035176,9.690752e+02
4083388403051261561560495289181218537493-0,0.013598,-3.776774e+02
AMMIn,0.058779,1.627308e+03
AMMOut,-0.059071,-1.627308e+03
TOTAL NET,-0.000292,-1.037552e-07


In [114]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,WBTC-C599,USDC-eB48
332,6.041469e-03,-166.767446
4b7ef4b0433441ad854c728302248661,6.041469e-03,-166.767446
8ef933e3eef94f8ea2a73cc11aaa6fb7,-2.388722e-02,658.010325
d7bd6d60f8f14492a9c5587a56caf843,3.309805e-02,-916.102340
8748e770adfe49f78064fd9bcf72417e,-3.489210e-02,961.261703
4083388403051261561560495289181218537493-0,1.359833e-02,-377.679169
AMMIn,5.877932e-02,1619.272028
AMMOut,-5.877932e-02,-1627.316401
TOTAL NET,6.218137e-12,-8.044373


#### LYXe/USDC

In [115]:
pair = "LYXe-be6D/USDC-eB48"
pricedf.loc[Pair.n(pair)]

price           vl itm  bs  \
exchange   cid0                                       
carbon_v1  652071-1  15.999998  7503.700799       s   
uniswap_v3 558       14.391360     1.085336      bs   
           b787691f  14.391360     1.085336      bs   

                                                     bsv  
exchange   cid0                                           
carbon_v1  652071-1      sell-LYXe @ 16.00 USDC per LYXe  
uniswap_v3 558       buy-sell-LYXe @ 14.39 USDC per LYXe  
           b787691f  buy-sell-LYXe @ 14.39 USDC per LYXe

In [116]:
pi = CA.pair_data(pair)
O = CPCArbOptimizer(pi.CC)
r = O.margp_optimizer(pair.split("/")[0])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,LYXe-be6D,USDC-eB48
558,0.0,0.0
527756afcc054eb4961eb0ddb787691f,0.0,0.0
AMMIn,0.0,0.0
AMMOut,0.0,0.0
TOTAL NET,0.0,0.0


In [117]:
r = O.margp_optimizer(pair.split("/")[1])
r.trade_instructions(ti_format=O.TIF_DFAGGR)

,USDC-eB48,LYXe-be6D
558,6.821210e-13,-5.684342e-14
527756afcc054eb4961eb0ddb787691f,6.821210e-13,-5.684342e-14
AMMIn,1.364242e-12,0.000000e+00
AMMOut,0.000000e+00,-1.136868e-13
TOTAL NET,1.364242e-12,-1.136868e-13
